In [23]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [25]:
extracted_data=load_pdf_file(data='Data/')

In [17]:
#extracted_data

In [27]:
#split the data into text chunks

def split_text(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks



In [28]:
text_chunks=split_text(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  5860


In [21]:
#text_chunks

In [29]:
from langchain.embeddings import HuggingFaceEmbeddings

In [31]:
#Download the Embedding model from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [32]:
embeddings=download_hugging_face_embeddings()

C:\Users\Ansh\AppData\Local\Temp\ipykernel_6480\329743759.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [33]:
query_result = embeddings.embed_documents("Hello world")
print("Length of the query embedding:", len(query_result[0]))

Length of the query embedding: 384


In [63]:
#query_result

In [48]:
#load encironment variables
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [35]:

from pinecone import Pinecone, ServerlessSpec

pc= Pinecone(api_key=PINECONE_API_KEY)

index_name="curenow"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
)
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'ef0bd054939da1bfe4ba1f0b343c4ebe', 'Date': 'Fri, 07 Feb 2025 08:45:34 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [49]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [37]:
# Embed each chunk and upsert the embeddings into your Pinecone index
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [38]:
from langchain.vectorstores import Pinecone
#embed each chunk and upsert the embeddings into your Pinecone index
docsearch=Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [39]:
docsearch

In [40]:
retriever =docsearch.as_retriever(seach_type="similarity", search_kwargs={"k":3})

In [41]:
retrieved_docs= retriever.invoke("What is acne")

In [42]:
retrieved_docs

[Document(metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(metadata={'page': 38.0, 'page_label': '39', 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin d

In [43]:
pip install langchain-google-genai

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 8.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -------- ------------------------------- 2.6/12.8 MB 13.7 MB/s eta 0:00:01
   ------------- -------------------------- 4.5/12.8 MB 10.7 MB/s eta 0:00:01
   ----------------------- ---------------- 7.6/12.8 MB 12.4 MB/s eta 0:00:01
   --------------------------------- ------ 10.7/12.8 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 13.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------------------- ------------------ 2.4/4.3 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 12.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature= 0.5, 
    max_output_tokens=100,
    google_api_key=GEMINI_API_KEY
    )

In [57]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [59]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bone and soft tissue and other disturbances. When this abnormality occurs before bone growth stops, it results in unusual height, known as gigantism. Acromegaly is relatively rare, affecting approximately 50 out of every one million people.


In [1]:
print(len(text_chunks))  # This should show the total number of chunks from both PDFs


NameError: name 'text_chunks' is not defined